In [47]:
!pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain langchain chromadb sentence-transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [1]:
LANGCHAIN_TRACING_V2='true'
LANGCHAIN_ENDPOINT="https://api.smith.langchain.com"
LANGCHAIN_API_KEY="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"
LANGCHAIN_PROJECT="pr-smug-lung-72"

In [2]:
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
import bs4

# Load Documents
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)

# Load documents
docs = loader.load()

if docs:
    print(f"Loaded {len(docs)} documents")
    
    # Preview raw document content
    for i, doc in enumerate(docs[:1]):  # Preview the first document
        print(f"Raw Document {i}:\n{doc.page_content[:500]}...\n")
    
    # Safely clean the document content
    for doc in docs:
        # Check if content contains the target phrase for splitting
        if "Building agents with LLM" in doc.page_content:
            doc.page_content = doc.page_content.split("Building agents with LLM", 1)[-1]
        else:
            print("Warning: Target phrase not found. Keeping raw content.")
    
    # Preview cleaned content
    for i, doc in enumerate(docs[:1]):  # Preview the first cleaned document
        print(f"Cleaned Document {i}:\n{doc.page_content[:500]}...\n")
    
    # Split the documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    splits = text_splitter.split_documents(docs)
    
    print(f"Total document splits: {len(splits)}")
    
    # Convert splits to valid documents
    valid_documents = [
        Document(page_content=split.page_content, metadata=split.metadata) 
        for split in splits if split.page_content.strip()  # Ensure non-empty content
    ]
    
    print(f"Total valid document splits: {len(valid_documents)}")
    for i, doc in enumerate(valid_documents[:3]):  # Preview the first 3 valid splits
        print(f"Document chunk {i}:\n{doc.page_content[:200]}...\n")
else:
    print("No documents were loaded. Please check the webpage or selector.")


USER_AGENT environment variable not set, consider setting it to identify your requests.


Loaded 1 documents
Raw Document 0:


      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.
Agent System Overview#
In...

Cleaned Document 0:
 (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.
Agent System Overview#
In a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented

In [3]:
from sentence_transformers import SentenceTransformer
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.schema import Document

# Load embedding model
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Pass valid_documents (a list of LangChain Document objects) to Chroma
vectorstore = Chroma.from_documents(
    documents=valid_documents,  # List of Document objects
    embedding=embedding_model  # Pass the HuggingFaceEmbeddings object
)

print("Vector store created successfully!")


/home/drovco/miniconda3/envs/fusion/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_1312611/1748165121.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


Vector store created successfully!


In [4]:
# Example query text
query_text = "What are LLM-powered autonomous agents?"

# Generate an embedding for the query
query_embedding = embedding_model.embed_query(query_text)

# Perform the query on the vector store
results = vectorstore.similarity_search_with_score(
    query=query_text,
    k=5  # Number of top results to return
)

# Display the results
for idx, (doc, score) in enumerate(results):
    print(f"Result {idx+1}:")
    print(f"Content: {doc.page_content[:200]}")  # Preview first 200 characters
    print(f"Metadata: {doc.metadata}")
    print(f"Score: {score}")
    print("-" * 50)


Result 1:
Content: Fig. 1. Overview of a LLM-powered autonomous agent system.
Component One: Planning#
A complicated task usually involves many steps. An agent needs to know what they are and plan ahead.
Task Decomposit
Metadata: {'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}
Score: 0.6767662763595581
--------------------------------------------------
Result 2:
Content: (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extend
Metadata: {'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}
Score: 0.7238340973854065
--------------------------------------------------
Result 3:
Content: They also discussed the risks, especially with illicit drugs and bioweapons. They developed a test set containing a list of known chemical weapon agents and asked the agent to synthesize them. 4 out o
Metadata: {'source': 'https://li

In [5]:
from langchain.prompts import ChatPromptTemplate

# Prompt
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Answer the question based only on the following context:\n{context}\n\nQuestion: {question}\n'), additional_kwargs={})])

In [41]:
!pip install anthropic

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.5/199.5 kB 5.9 MB/s eta 0:00:00


In [47]:
from transformers import pipeline

# Load the model and tokenizer
model = pipeline('text-generation', model='gpt2')

from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

# Prompt
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Answer the question based only on the following context:\n{context}\n\nQuestion: {question}\n'), additional_kwargs={})])

In [48]:
chain = prompt | model


In [73]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import HuggingFacePipeline
from transformers import pipeline

# Define the prompt template with placeholders for context and question
prompt_template = "Context: {context}\nQuestion: {question}\nAnswer:"

# Initialize the Hugging Face text generation pipeline
model = pipeline(
    'text-generation',
    model='gpt2',
    max_length=1024,  # Maximum length for input + output
    pad_token_id=50256,  # Prevents errors for padding
)

# Wrap the Hugging Face pipeline in a HuggingFacePipeline (LangChain's Runnable)
llm = HuggingFacePipeline(pipeline=model)

# Create the prompt template
prompt = PromptTemplate(input_variables=["context", "question"], template=prompt_template)

# Create the LLMChain with the wrapped LLM
llm_chain = LLMChain(prompt=prompt, llm=llm)

# Truncate the context to fit within the token limit
context = "Truncated documents context goes here..."  # Ensure this is the actual content from `docs`

# Define the input variables
inputs = {
    "context": context,  # Truncated documents context
    "question": "What is Task Decomposition?"
}

# Run the chain
response = llm_chain.run(context=inputs["context"], question=inputs["question"])

print(response)


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Context: Truncated documents context goes here...
Question: What is Task Decomposition?
Answer: Task deconposition is based on the task (s:tasks, t:Tasks). It can take either an object or its root (object, t:Tasks), or a list of tasks that are in turn ordered by its context.
The concept is that an object gets its context before an object can have one, if the context of its root has some other properties of its type as shown in this excerpt from Heterogeneous.
Question: What are the differences between an object and a list?
Answer: When you try it, it is hard to see how they can be combined. Here are two examples; I want to show what is happening when you try it.
(a) An object contains the list(s); while the process of deconstructing a list does not, it can still be viewed as taking time and time to decompress itself by some other time point. This has to do with how it is represented in an HUnit file. If you want to use a list for the rest of your task, then the same process, that has a